In [1]:
import pandas as pd
from typing import Literal
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from utils.loader import MovieLensDataset
from arquitecture.Recommender import Recommender
from arquitecture.components.PatterAnalyzer import PatternAnalyzer
import torch.optim as optim

SEED = 55
BATCH = 5000
NUN_THREADS = 6
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCH = 400
LEARNING_RATE = 0.00001

In [2]:
dataset = MovieLensDataset(ml_path="ml-100k", split="test", seed=SEED)
user_data, train_tensor, test_tensor = dataset.__getitem__(11)
test_tensor

tensor([1.0000, 0.5024, 0.1627, 0.1675, 0.9378, 0.2871, 0.0048, 0.6986, 0.0144,
        0.0813, 0.1675, 0.1340, 0.1722, 0.3780, 0.4785, 0.7225, 0.2297, 0.1053,
        0.0000])

In [3]:
print(f"user_data shape: {user_data.size()}")
print(f"train_tensor shape: {train_tensor.size()}")
print(f"test_tensor shape: {test_tensor.size()}")

user_data shape: torch.Size([23])
train_tensor shape: torch.Size([19, 22])
test_tensor shape: torch.Size([19])


In [4]:
train_dataset = MovieLensDataset(ml_path="ml-100k", split="train", seed=SEED)
test_dataset = MovieLensDataset(ml_path="ml-100k", split="test", seed=SEED)
val_dataset = MovieLensDataset(ml_path="ml-100k", split="val", seed=SEED)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)

try:
    for i, batch in enumerate(train_dataloader):
        user_data_tensor, rating_train_tensor, rating_test_tensor = batch
        print(f"Batch {i}:")
        print("  user_data_tensor shape:", user_data_tensor.shape)
        print("  rating_train_tensor shape:", rating_train_tensor.shape)
        print("  rating_test_tensor shape:", rating_test_tensor.shape)
        # Solo mostramos el primer batch
        break
except Exception as e:
    print("Error al iterar sobre el DataLoader:", e)

Batch 0:
  user_data_tensor shape: torch.Size([678, 23])
  rating_train_tensor shape: torch.Size([678, 19, 22])
  rating_test_tensor shape: torch.Size([678, 19])


In [5]:
height=19
width=22
user_input_size = 23
    # Crear instancia del modelo
pattern_analyzer = PatternAnalyzer(conv_structure=[1,4,8,16,32,32,32],
                            input_size=torch.Size((height, width)),
                            pool_depth=3,
                            expert_hidden_size=16,
                            expert_output_len=4,
                            final_mlp_factor=2,
                            final_mlp_output_len=15,
                            ).to(DEVICE)
    
model = Recommender(
        user_data_input_size=user_input_size,
        user_data_analizer_factor = 2,
        user_data_analizer_output_size = 10,
        pattern_analyzer=pattern_analyzer,
        final_regressor_factor=1,
        final_regressor_output_len=19
    ).to(DEVICE)

ratings = torch.randn(628, height, width).to(DEVICE)
user_data = torch.randn(628, user_input_size).to(DEVICE)
    

    # Pasar el tensor por el modelo
output = model(user_data, ratings)
print(output)
    # Mostrar la forma de la salida final
print("Forma de la salida después de `view`:", output.shape)
print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

tensor([[[[-0.1281,  0.6064, -0.4078,  ..., -1.6076, -1.1058,  1.4426],
          [ 0.6030, -1.1032,  0.1374,  ..., -0.2090,  1.3200,  1.3305],
          [ 0.7843,  1.0782, -0.7403,  ...,  1.0806,  0.4374, -0.1697],
          ...,
          [-1.4503, -1.1877,  0.8442,  ...,  0.8604,  1.2604,  1.3787],
          [-0.7359, -0.2517,  0.2049,  ...,  1.6210, -0.3067,  0.6239],
          [ 1.9142,  1.0068, -0.4572,  ..., -1.2460, -0.3769,  1.5491]]],


        [[[-0.4804,  1.8881, -0.6675,  ...,  0.5989, -1.7020, -2.2787],
          [-1.2628,  1.3608, -0.3817,  ..., -2.2630, -0.9694,  0.4777],
          [-0.5104, -0.1009,  0.2481,  ...,  0.4398,  1.2223,  0.8874],
          ...,
          [ 0.5461, -1.0378,  1.2874,  ...,  0.2650, -0.4102,  2.1520],
          [ 0.8942, -2.2010,  1.4985,  ..., -1.2943,  0.2081, -0.0424],
          [ 1.0874,  0.8067,  1.1076,  ..., -1.0154,  0.2729,  1.0720]]],


        [[[ 2.1588,  0.7361, -0.9114,  ..., -1.7064,  0.2851, -0.4459],
          [ 0.6465,  0.769

In [6]:
# Crear el optimizador, por ejemplo, usando Adam con una tasa de aprendizaje de 0.001
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Crear la función de pérdida para regresión (Mean Squared Error Loss)
criterion = nn.MSELoss(reduction="sum")

In [7]:
for epoch in range(NUM_EPOCH):
    # --- Training Phase ---
    model.train()  # Set model to training mode
    running_train_loss = 0.0
    
    for user_data_tensor, rating_train_tensor, rating_test_tensor in train_dataloader:
        optimizer.zero_grad()   
             
        outputs = model(user_data_tensor.to(DEVICE), rating_train_tensor.to(DEVICE)).to(DEVICE)        
        
        loss = criterion(outputs.to(DEVICE), rating_test_tensor.to(DEVICE)) 
        
        loss.backward()                 # Backpropagation
        optimizer.step()                # Update model parameters
        running_train_loss += loss.item() * rating_test_tensor.size(0)
    
    epoch_train_loss = running_train_loss / len(train_dataloader.dataset)
    
    # --- Validation Phase ---
    model.eval()  # Set model to evaluation mode
    running_val_loss = 0.0
    with torch.no_grad():
        for user_data_tensor, rating_train_tensor, rating_test_tensor in val_dataloader:
            
            outputs = model(user_data_tensor.to(DEVICE), rating_train_tensor.to(DEVICE))        
            loss = criterion(outputs, rating_test_tensor.to(DEVICE)) 
            
            running_val_loss += loss.item() * rating_test_tensor.size(0)
    
    epoch_val_loss = running_val_loss / len(val_dataloader.dataset)
    
    print(f"Epoch [{epoch+1}/{NUM_EPOCH}] - Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")


tensor([[[[5.0000e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 1.2676e-02],
          [5.0000e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 3.2938e-03],
          [7.5000e-01, 5.9142e-04, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 8.4445e-02],
          ...,
          [5.0000e-01, 3.5532e-03, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 1.5622e-01],
          [5.0000e-01, 4.0144e-03, 0.0000e+00,  ..., 0.0000e+00,
           1.0000e+00, 1.0243e-01],
          [7.5000e-01, 4.0144e-03, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 6.6517e-02]]],


        [[[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00,
           0.0000e+00, 3.1564e-02],
          [1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 7.1734e-03],
          [1.0000e+00, 2.0919e-02, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 2.6747e-02],
          ...,
          [1.0000e+00, 4.6203e-01, 0.0000e+00,  ..., 0.00

KeyboardInterrupt: 